# Scraper

크롤 에이전트

| **Data Scraping** | **Data Crawling** |
| :-: | :-: |
| 웹을 포함한 다양한 소스들을 추출 | 웹을 돌아다니며 링크들을 찾아 탐색 |
| 어느 사이즈에서도 가능 | 보통 대규모로 진행 |
| 필수는 아님 | 중복 제거 필수 |
| 크롤 에이전트 & parser 필요 | 크롤 에이전트 필요 |

In [1]:
from requests import request, get
from requests.compat import urljoin, urlparse, urlunparse
from bs4 import BeautifulSoup
import re

### 국내 홈페이지에서의 문제점: 뽐뿌

In [19]:
headers = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Whale/3.26.244.21 Safari/537.36'}

url = 'https://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1735&no=9297710'
resp = request('get', url)
resp.status_code, resp.headers['content-type']

(200, 'text/html; charset=euc-kr')

In [ ]:
# 우리나라에서밖에 못 쓰는 euc-kr, 굉장히 옛날 것...
# euc-kr    ===>    Unicode
# 완성형    확장완성형   유니코드
# windows98        | 모두 해결
# (코드부여)
# 가 = 1
# 나 = 2
# 쀍 = X (안쓰는 것들은..)
# 해외에서 사용 불가, 1음절 = 11,172자 모두 표현 불가, 순서가 제멋대로

In [3]:
dom = BeautifulSoup(resp.text, 'html.parser')

In [5]:
resp.encoding

'euc-kr'

In [7]:
resp.encoding = 'utf8'

In [8]:
resp.text

'\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ko" xml:lang="ko">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr" />\n<meta http-equiv="Content-Script-Type" content="text/javascript" />\n<meta http-equiv="Content-Style-Type" content="text/css" />\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\n\n\n<meta property="og:site_name" content="�˻�" />\n<meta property="og:type" content="article" />\n<meta property="og:url" content="https://www.ppomppu.co.kr/zboard/view.php?id=freeboard&no=9297710" />\n<meta property="og:title" content="������ �������� �ظ��̳� �̤�" />\n<meta property="og:description" content="���� ���� ���� ó���ε� �ƹ��� �����̶����� �̰� �� �̤�\'���� �ӾҼ���\' ���µ� ������ �� ����" />\n<meta name="description" content="���� ���� ���� ó���ε� �ƹ��� �����̶����� �̰� �� �̤�\'���� �ӾҼ���\' ���µ� ������ �� ����" />\n<meta pro

In [13]:
dom = BeautifulSoup(resp.text, 'html.parser') # 가져오지 못함
dom.select_one('.board-contents')

In [14]:
dom = BeautifulSoup(resp.text, 'lxml') # html5lib는 가능
dom.select_one('.board-contents')

<td align="left" class="board-contents" valign="top">
<p>���� ���� ���� ó���ε� �ƹ��� �����̶����� �̰� �� �̤�</p><p> </p><p>'���� �ӾҼ���' ���µ� ������ �� ���׿�</p><p> </p><p>���� �׷�����??? ������</p><p> </p><p> </p><!--"&lt;--></td>

In [16]:
url = 'https://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1735&no=9297710'
resp = request('get', url)
dom = BeautifulSoup(resp.text, 'html.parser')
dom.select_one('.board-contents')

In [17]:
resp.text # board-contents가 있는데도 가져오지를 못함.

'\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ko" xml:lang="ko">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr" />\n<meta http-equiv="Content-Script-Type" content="text/javascript" />\n<meta http-equiv="Content-Style-Type" content="text/css" />\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\n\n\n<meta property="og:site_name" content="뽐뿌" />\n<meta property="og:type" content="article" />\n<meta property="og:url" content="https://www.ppomppu.co.kr/zboard/view.php?id=freeboard&no=9297710" />\n<meta property="og:title" content="아이유 고딩역이 왠말이냐 ㅜㅜ" />\n<meta property="og:description" content="나이 서른 넘은 처자인데 아무리 동안이라지만 이건 좀 ㅜㅜ\'폭싹 속았수다\' 보는데 집중이 팍 깨네" />\n<meta name="description" content="나이 서른 넘은 처자인데 아무리 동안이라지만 이건 좀 ㅜㅜ\'폭싹 속았수다\' 보는데 집중이 팍 깨네" />\n<meta property="og:image" content="https://www.ppomppu.co.kr/images/icon_app_201

In [18]:
dom = BeautifulSoup(resp.text, 'html.parser')
dom = BeautifulSoup(resp.text, 'lxml')
dom = BeautifulSoup(resp.text, 'html5lib')
dom.select_one('.board-contents') #####
# DOM을 통해 탐색할 때, 구조와 속성이 중요한 이유
# 1. Parser의 성능이 다름
# 2. 사이트를 만든 사람을 믿을 수 없기 때문

<td align="left" class="board-contents" valign="top">
<p>나이 서른 넘은 처자인데 아무리 동안이라지만 이건 좀 ㅜㅜ</p><p> </p><p>'폭싹 속았수다' 보는데 집중이 팍 깨네요</p><p> </p><p>저만 그런가요??? ㄷㄷㄷ</p><p> </p><p> </p><!--"&lt;--></td>

```html
<td class='board-contents' align="left" valign=top class=han>
<td class="board-contents" align="left" valign=top class=han>
```
a = han

a = board-contents

a => ?

### 네이버웹툰에서 문제

In [20]:
url = 'https://comic.naver.com/index'
resp = request('GET', url, headers=headers)
resp.status_code, resp.headers['content-type']

(200, 'text/html;charset=UTF-8')

In [23]:
dom = BeautifulSoup(resp.text, 'html.parser')
dom
# body가 비어있음 => Asynchronous JAvascript & Xml = AJAX(Dynamic HTML)
# <body>
# <div id="root"></div>
# </body>


<!DOCTYPE html>

<html lang="ko">
<head>
<title>네이버 웹툰</title>
<link href="https://shared-comic.pstatic.net/favicon/favicon_96x96.ico" rel="icon" type="image/x-icon"/>
<link href="https://m.comic.naver.com/index" rel="canonical">
<meta content="6Wv2YrpTSpapViVikuUS-ebAqZDiKxcrDV_clkRzH9A" name="google-site-verification"/>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="article" property="og:type"/>
<meta content="네이버 웹툰" property="og:article:author"/>
<meta content="https://comic.naver.com" property="og:article:author:url"/>
<meta content="" property="og:title"/>
<meta content="https://ssl.pstatic.net/static/comic/images/og_tag_v3.png" property="og:image"/>
<meta content="매일매일 새로운 재미, 네이버 웹툰." property="og:description"/>
<script>
		if (/MSIE \d|Trident.*rv:/.test(navigator.userAgent)) {
			window.location = 'microsoft-edge:' + window.location;
			setTimeout(function () {
				window.location = 'https://go.microsoft.com/fwlink/?linkid=213554

In [27]:
URLs = ['https://www.scrapingcourse.com/pagination']
Seens = []

while URLs:
  seed = URLs.pop(0) # 하나씩 접근하므로 queue가 적합할 듯
  Seens.append(seed)

  # robots.txt 확인
  print(seed)

  resp = request('GET', seed)

  if resp.status_code != 200:
      continue

  if not re.search(r'text/html', resp.headers['content-type']):
      continue

  dom = BeautifulSoup(resp.text, 'html.parser')

  for a in dom.select('a[href]'):
      link = a.attrs['href']
      nurl = urljoin(seed, link)
      if re.match(r'https://www.scrapingcourse.com/pagination', nurl):
          # Focused: 도메인
          if nurl not in URLs and nurl not in Seens:
              URLs.append(nurl)

https://www.scrapingcourse.com/pagination
https://www.scrapingcourse.com/pagination/2
https://www.scrapingcourse.com/pagination/3
https://www.scrapingcourse.com/pagination/13
https://www.scrapingcourse.com/pagination/1
https://www.scrapingcourse.com/pagination/4
https://www.scrapingcourse.com/pagination/5
https://www.scrapingcourse.com/pagination/12
https://www.scrapingcourse.com/pagination/11
https://www.scrapingcourse.com/pagination/6
https://www.scrapingcourse.com/pagination/7
https://www.scrapingcourse.com/pagination/10
https://www.scrapingcourse.com/pagination/9
https://www.scrapingcourse.com/pagination/8


In [29]:
# 이미지도 함께 수집하도록 수정
URLs = ['https://www.scrapingcourse.com/pagination']
Seens = []

while URLs:
  seed = URLs.pop(0) # 하나씩 접근하므로 queue가 적합할 듯
  Seens.append(seed)

  # robots.txt 확인
  print(seed)

  resp = request('GET', seed)

  if resp.status_code != 200:
      continue

  # if not re.search(r'text/html', resp.headers['content-type']):
  #     continue

  # 링크+본문 Scraping 영역
  if re.search(r'image/(?:jpeg|jpg|gif|png|bmp)', resp.headers['content-type']):
  # image
      name = re.sub(r'[?:;/$]', '', seed['url'])
      ext = re.search(r'image/(jpeg|jpg|gif|png|bmp)', resp.headers['content-type']).group(1)
      with open(name+'.'+ext, 'wb') as fp:
          fp.write(resp.content)

  # 각 페이지의 이미지도 함께 수집
  # headers['content-type']
  # dom.select('a')
  # Focused: 전략 세워야 함
  elif re.search(r'text/html', resp.headers['content-type']):

    dom = BeautifulSoup(resp.text, 'html.parser')

    for a in dom.select('a[href]'):
        link = a.attrs['href']
        nurl = urljoin(seed, link)
        if re.match(r'https://www.scrapingcourse.com/pagination', nurl):
            # Focused: 도메인
            if nurl not in URLs and nurl not in Seens:
                URLs.append(nurl)

https://www.scrapingcourse.com/pagination
https://www.scrapingcourse.com/pagination/2
https://www.scrapingcourse.com/pagination/3
https://www.scrapingcourse.com/pagination/13
https://www.scrapingcourse.com/pagination/1
https://www.scrapingcourse.com/pagination/4
https://www.scrapingcourse.com/pagination/5
https://www.scrapingcourse.com/pagination/12
https://www.scrapingcourse.com/pagination/11
https://www.scrapingcourse.com/pagination/6
https://www.scrapingcourse.com/pagination/7
https://www.scrapingcourse.com/pagination/10
https://www.scrapingcourse.com/pagination/9
https://www.scrapingcourse.com/pagination/8
